# Toscanini BYOC — Rosetta FastRelax
**Audit ID:** `REPLACE_WITH_YOUR_AUDIT_ID`  
**Verdict:** `VETO`  
**Violations:** `LAW-125, LAW-130`  
**Platform:** Google Colab (NVIDIA T4)

> ⚠️ **Rosetta requires a license** (free for academic use).  
> Register at: https://www.rosettacommons.org/software/license-and-download  
> Upload your Rosetta binary to this session before running.


## Step 1: Configure Rosetta Path

In [ ]:
import os

# Set the path to your Rosetta binary
# Upload it to this session or mount from Google Drive
ROSETTA_BIN = "/content/rosetta_scripts.default.linuxgccrelease"  # Colab
# ROSETTA_BIN = "/kaggle/working/rosetta_scripts.default.linuxgccrelease"  # Kaggle

PDB_FILE   = "your_structure.pdb.pdb"
OUTPUT_PDB = "your_structure.pdb_relaxed.pdb"

assert os.path.exists(ROSETTA_BIN), f"Rosetta binary not found: {ROSETTA_BIN}"
print(f"Rosetta: {ROSETTA_BIN}")
print(f"Input:   {PDB_FILE}")

## Step 2: Generate FastRelax Protocol

In [ ]:
XML_PROTOCOL = '''<ROSETTASCRIPTS>
  <SCOREFXNS>
    <ScoreFunction name="toscanini_relax" weights="ref2015">
      <Reweight scoretype="rama_prepro" weight="2.0"/>
      <Reweight scoretype="fa_dun"      weight="1.0"/>
      <Reweight scoretype="fa_rep"      weight="1.5"/>
      <Reweight scoretype="omega"       weight="1.0"/>
      <Reweight scoretype="dslf_fa13"   weight="1.0"/>
      <Reweight scoretype="cart_bonded" weight="1.0"/>
    </ScoreFunction>
  </SCOREFXNS>
  <MOVERS>
    <FastRelax name="relax" scorefxn="toscanini_relax" repeats="5"/>
    <MinMover  name="min"   scorefxn="toscanini_relax"
               type="lbfgs_armijo_nonmonotone" tolerance="0.001" max_iter="200"/>
  </MOVERS>
  <PROTOCOLS>
    <Add mover="relax"/>
    <Add mover="min"/>
  </PROTOCOLS>
</ROSETTASCRIPTS>'''

with open("protocol.xml", "w") as f:
    f.write(XML_PROTOCOL)
print("Protocol written: protocol.xml")

## Step 3: Run FastRelax

In [ ]:
import subprocess

cmd = [
    ROSETTA_BIN,
    "-in:file:s",      PDB_FILE,
    "-parser:protocol","protocol.xml",
    "-out:file:o",     OUTPUT_PDB,
    "-out:nstruct",    "1",
    "-out:overwrite",
    "-ex1", "-ex2",
    "-use_input_sc",
    "-mute",           "all",
    "-unmute",         "protocols.relax",
]

print("Running Rosetta FastRelax...")
result = subprocess.run(cmd, capture_output=True, text=True, timeout=3600)

if result.returncode == 0:
    print("✅ FastRelax complete")
else:
    print(f"⚠️ Rosetta exited {result.returncode}")
    print(result.stderr[-500:])

## Step 4: Post Results to Toscanini

In [ ]:
import requests

CALLBACK_TOKEN = "PASTE_YOUR_CALLBACK_TOKEN_HERE"
BACKEND_URL    = "http://localhost:8000"

with open(OUTPUT_PDB, "rb") as f:
    response = requests.post(
        f"{BACKEND_URL}/refinement/callback",
        data={"token": CALLBACK_TOKEN},
        files={"file": (OUTPUT_PDB, f, "chemical/x-pdb")},
        timeout=120
    )

if response.status_code == 200:
    result = response.json()
    print("✅ Re-certification complete!")
    print(f"   Refined audit  : {result.get('refined_audit_id')}")
    print(f"   New verdict    : {result.get('verdict')}")
    print(f"   Coverage       : {result.get('coverage_pct')}%")
else:
    print(f"⚠️ Callback failed: {response.text[:200]}")
